**Dependencies**

In [2]:
import cv2
import os
import numpy as np
import shutil
import random

from pathlib import Path
from ultralytics import YOLO


**List of all the models**

In [ ]:
# MODELS
# ..\\main\\model\\plaque_model_seg\\weights\\best.pt (Plaque Psoriasis Segmentation Model)
# ..\\main\\model\\atopic_model_seg\\weights\\best.pt (Atopic Dermatitis Segmentation Model)
# ..\main\ultralytics\ultralytics\cfg\models\v8\yolov8-cls-resnet50.yaml (Location of the Resnet50 Backbone)
# .. main\\model\\resnet_mix_ep50\\weights\\best.pt (Classification 50 Epoch)
# .. main\\model\\resnet_mix_ep100\\weights\\best.pt (Classification 100 Epoch)

**Image Processing**
> This are includes the bicubic, segmentation, and resizing.

Run this only if you are pre-processing a batch of images

In [29]:
# Specify the folder paths
in_folder_path = r'C:\Users\Leandro\temp\main\train\images\test\Yellowtail_Scad'
out_folder_path = r'C:\Users\Leandro\temp\main\test\test\Yellowtail_Scad'

# Create the output folder if it doesn't exist
os.makedirs(out_folder_path, exist_ok=True)

# Iterate over files in the input folder
for filename in os.listdir(in_folder_path):
    # Construct the full file path
    file_path = os.path.join(in_folder_path, filename)

    # Read the image
    img = cv2.imread(file_path)
    if img is not None:
        # Check if the image is successfully read
        # Resize the image to 128x128 using bicubic interpolation
        img_resized = cv2.resize(img, (128, 128), interpolation=cv2.INTER_CUBIC)

        # Get the base name of the file without the extension
        base_name = os.path.splitext(filename)[0]

        # Construct the output file path with the .png extension
        out_file_path = os.path.join(out_folder_path, f"{base_name}.png")

        # Save the resized image to the output folder
        cv2.imwrite(out_file_path, img_resized)
        print(f"Resized and saved: {out_file_path}")
    else:
        print(f"Unable to read image: {file_path}")

print('Completed!')

Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\Image_11.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\Image_2.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\Image_3.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\Image_5.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\Image_8.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\IMG20240518073801_BURST012.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\IMG20240518073801_BURST018.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\IMG20240518073822_BURST006.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\IMG20240518073822_BURST007.png
Resized and saved: C:\Users\Leandro\temp\main\test\test\Yellowstripe_Scad\IMG20240518073822_BURST009.png
Resized and saved: C:\Users\Leandro\temp\mai

Run this only if you are pre-processing a single image

In [36]:
# Read the image
img = cv2.imread(r'C:\Users\Leandro\temp\main\test\test_1.png')

# Resize the image to 128x128 using bicubic interpolation
img_resized = cv2.resize(img, (128, 128), interpolation=cv2.INTER_CUBIC)

# Display the resized image
cv2.imshow('Resized Image', img_resized)
cv2.waitKey(0)  # Wait for any key press
cv2.destroyAllWindows()  # Close all OpenCV windows

**Image Segmentation - PLAQUE**

In [ ]:
# Load the YOLO model
model = YOLO(r'C:\\Users\\Leandro\\temp\\main\\model\\plaque_model_seg\\weights\\best.pt', task='detect')

# Specify the input folder
input_folder = r'C:\\Users\\Leandro\\temp\\main\\data\\bicubic_plaque_128' #Make sure to change this value

# Specify the output folder
output_folder = r'C:\\Users\\Leandro\\temp\\main\\data\\orig_seg_plaque' #Make sure to change this value
os.makedirs(output_folder, exist_ok=True)

# Iterate over files in the input folder
for filename in os.listdir(input_folder):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the full file path
        image_path = os.path.join(input_folder, filename)

        # Read the image
        img = cv2.imread(image_path)

        # Get the dimensions of the image
        H, W, _ = img.shape

        # Run the YOLO model on the image
        results = model.predict(img, device='cpu')

        # Check if any objects were detected
        if results is not None and len(results) > 0:
            for result in results:
                # Check if the result has masks
                if result.masks is not None:
                    for j, mask in enumerate(result.masks.data):
                        mask = mask.numpy() * 255
                        mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)  # Use nearest neighbor interpolation

                        # Calculate the area of the detected object
                        object_area = np.sum(mask > 0)
                        image_area = W * H

                        # Calculate the percentage of the image area covered by the object
                        object_percentage = (object_area / image_area) * 100
                        if object_percentage >= 10:
                            # Segment the image
                            segmented_img = cv2.bitwise_and(img, img, mask=mask.astype(np.uint8))

                            # Resize the segmented image to 128x128
                            segmented_img = cv2.resize(segmented_img, (128, 128), interpolation=cv2.INTER_AREA)

                            # Save the segmented image
                            output_path = os.path.join(output_folder, f'{filename}')
                            cv2.imwrite(output_path, segmented_img)
                        else:
                            # Resize the original image to 128x128
                            resized_img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)

                            # Save the resized image
                            output_path = os.path.join(output_folder, filename)
                            cv2.imwrite(output_path, resized_img)

**Image Segmentation - ATOPIC**

In [ ]:
# Load the YOLO model
model = YOLO(r'C:\\Users\\Leandro\\temp\\main\\model\\atopic_model_seg\\weights\\best.pt', task='detect')

# Specify the input folder
input_folder = r'C:\\Users\\Leandro\\temp\\main\\data\\bicubic_atopic_128'

# Specify the output folder
output_folder = r'C:\\Users\\Leandro\\temp\\main\\data\\orig_seg_atopic'
os.makedirs(output_folder, exist_ok=True)

# Iterate over files in the input folder
for filename in os.listdir(input_folder):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the full file path
        image_path = os.path.join(input_folder, filename)

        # Read the image
        img = cv2.imread(image_path)

        # Get the dimensions of the image
        H, W, _ = img.shape

        # Run the YOLO model on the image
        results = model.predict(img, device='cpu')

        # Check if any objects were detected
        if results is not None and len(results) > 0:
            for result in results:
                # Check if the result has masks
                if result.masks is not None:
                    for j, mask in enumerate(result.masks.data):
                        mask = mask.numpy() * 255
                        mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)  # Use nearest neighbor interpolation

                        # Calculate the area of the detected object
                        object_area = np.sum(mask > 0)
                        image_area = W * H

                        # Calculate the percentage of the image area covered by the object
                        object_percentage = (object_area / image_area) * 100
                        if object_percentage >= 10:
                            # Segment the image
                            segmented_img = cv2.bitwise_and(img, img, mask=mask.astype(np.uint8))

                            # Resize the segmented image to 128x128
                            segmented_img = cv2.resize(segmented_img, (128, 128), interpolation=cv2.INTER_AREA)

                            # Save the segmented image
                            output_path = os.path.join(output_folder, f'{filename}')
                            cv2.imwrite(output_path, segmented_img)
                        else:
                            # Resize the original image to 128x128
                            resized_img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)

                            # Save the resized image
                            output_path = os.path.join(output_folder, filename)
                            cv2.imwrite(output_path, resized_img)

**This is for splitting the images in 80 20 ratio**

In [5]:
# Input directories
seg_atopic_dir = Path(r'C:\Users\Leandro\temp\main\data\true_data_atopic')
seg_plaque_dir = Path(r'C:\Users\Leandro\temp\main\data\true_data_plaque')
# Output directory
output_dir = Path('C:\\Users\\Leandro\\temp\\main\\train\\data')

# Create output directories
train_dir = output_dir / 'train'
val_dir = output_dir / 'val'

train_dir.mkdir(parents=True, exist_ok=True)
val_dir.mkdir(parents=True, exist_ok=True)

train_plaque_dir = train_dir / 'plaque'
train_atopic_dir = train_dir / 'atopic'
val_plaque_dir = val_dir / 'plaque'
val_atopic_dir = val_dir / 'atopic'

train_plaque_dir.mkdir(exist_ok=True)
train_atopic_dir.mkdir(exist_ok=True)
val_plaque_dir.mkdir(exist_ok=True)
val_atopic_dir.mkdir(exist_ok=True)

# Collect all image files
atopic_files = [f for f in seg_atopic_dir.glob('*') if f.is_file()]
plaque_files = [f for f in seg_plaque_dir.glob('*') if f.is_file()]

# Shuffle the files
random.shuffle(atopic_files)
random.shuffle(plaque_files)

# Split into 80% train and 20% validation
num_train_atopic = int(0.8 * len(atopic_files))
num_train_plaque = int(0.8 * len(plaque_files))

train_atopic_files = atopic_files[:num_train_atopic]
val_atopic_files = atopic_files[num_train_atopic:]

train_plaque_files = plaque_files[:num_train_plaque]
val_plaque_files = plaque_files[num_train_plaque:]

# Copy files to respective directories
for file in train_atopic_files:
    shutil.copy(file, train_atopic_dir)
for file in val_atopic_files:
    shutil.copy(file, val_atopic_dir)
for file in train_plaque_files:
    shutil.copy(file, train_plaque_dir)
for file in val_plaque_files:
    shutil.copy(file, val_plaque_dir)

print("Files copied successfully!")

Files copied successfully!


**Training script for resnet50**

In [ ]:
os.environ['WANDB_DISABLED'] = 'true'

# Load a model
model = YOLO(r'C:\Users\Leandro\temp\main\ultralytics\ultralytics\cfg\models\v8\yolov8-cls.yaml')  # build a new model from YAML

# Train the model
results = model.train(data=r'C:\Users\Leandro\temp\main\train\data\images', epochs=50, imgsz=128, task='classify', device='cpu', batch=32, augment=True)

**Predicting base on the results of training using the best model**

In [ ]:
from ultralytics import YOLO
import numpy
import cv2

# Load a pretrained YOLOv8n model
model = YOLO(r'C:\Users\Leandro\temp\main\model\resnet_true\weights\last.pt')
# Read the image
img = cv2.imread('C:\\Users\\Leandro\\temp\\main\\test\\test_anan.png')

# Resize the image to 128x128 using bicubic interpolation
img_resized = cv2.resize(img, (128, 128), interpolation=cv2.INTER_CUBIC)
# Display the resized image
cv2.imshow('Resized Image', img_resized)
cv2.waitKey(0)  # Wait for any key press
cv2.destroyAllWindows()  # Close all OpenCV windows

# Run inference on 'bus.jpg' with arguments
results = model.predict(img_resized, save=False, imgsz=128, conf=0.85, show=True)
threshold = 0.85
# Get the class names from the model
class_names = model.model.names

for result in results:
    for result in results:
        score = result.probs.top1conf
        prediction = result.probs.data.cpu().numpy()
    if score < threshold:
        print(f"Can't classify skin disease")
    else:
        print(f"Classified with a confidence score of {score * 100:.2f}%")
        # Iterate over class indices and probabilities
        for i, prediction in enumerate(prediction):
            class_name = class_names[i]
            print(f"{class_name}: {prediction * 100:.2f}%")